In [1]:
from math import sqrt
import numpy as np
import scipy
from scipy.stats import spearmanr
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from scipy.spatial.distance import cosine
import pandas as pd
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.isotonic import IsotonicRegression
import tensorflow as tf
import tensorflow_hub as hub
from sklearn.model_selection import train_test_split
import pickle
import string
from rake_nltk import Rake
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aruni\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\aruni\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
import warnings
warnings.filterwarnings('ignore')

## Evaluation Metrics

In [3]:
# Metrics functions : rmse and pearson correlation
def RMSE(actual, pred):
    return sqrt(mean_squared_error(actual, pred))

def Pearson(actual,pred):
    mean_a = sum(actual) / len(actual)
    mean_p = sum(pred) / len(pred)
    cov = sum((a - mean_a) * (b - mean_p) for (a, b) in zip(actual, pred)) / len(actual)
    p = float(cov / (np.std(actual) * np.std(pred)))
    return p

## Regression Models

In [11]:
def linear_reg(xTrain,yTrain,xTest):
    model = LinearRegression()
    model.fit(xTrain, yTrain)
    y_pred = model.predict(xTest)
    y_pred = np.clip(y_pred, 0, 5)
    return y_pred

def isotonic_reg(xTrain,yTrain,xTest):
    model = IsotonicRegression()
    model.fit(xTrain, yTrain)
    y_pred = model.predict(xTest)
    y_pred = np.clip(y_pred, 0, 5)
    return y_pred

def ridge_reg(xTrain, yTrain, xTest):
    model = Ridge()
    model.fit(xTrain, yTrain)
    y_pred = model.predict(xTest)
    y_pred = np.clip(y_pred, 0, 5)
    return y_pred

## Evaluation
The similarity scores are used as data to train different regression models.

In [7]:
def avg(l):
    return sum(l) / len(l)

def check_nan(arr):
    idx_NaN = np.isnan(arr)
    arr[idx_NaN] = 0
    return arr

In [8]:
def evaluate_models(mdl, df):
    mdl_score = 'normalized_' + mdl + '_similarity_score'
    X=df[[mdl_score,'keyword_match','normalized_length_ratio']]
    X_iso=df[mdl_score]
    y=df['score_avg']
    
    xTrain, xTest, yTrain, yTest = train_test_split(X, y, test_size=0.3, random_state=101)
    xTrain_i, xTest_i, yTrain_i, yTest_i = train_test_split(X_iso, y, test_size=0.3, random_state=101)


    linear_yPred = [float(x) for x in linear_reg(xTrain,yTrain,xTest)]
    ridge_yPred = [float(x) for x in ridge_reg(xTrain,yTrain,xTest)]
    isotonic_yPred = list(np.nan_to_num(isotonic_reg(xTrain_i,yTrain_i,xTest_i), nan=0))

    y = check_nan(np.asarray(yTest))
    y_iso = check_nan(np.asarray(yTest_i))
    isotonic_pred = check_nan(np.asarray([round(i*2)/2  for i in isotonic_yPred]))
    linear_pred = check_nan(np.asarray([round(i*2)/2  for i in linear_yPred]))
    ridge_pred = check_nan(np.asarray([round(i*2)/2  for i in ridge_yPred]))

    return RMSE(y_iso,isotonic_pred), Pearson(y_iso,isotonic_pred), RMSE(y,linear_pred), Pearson(y,linear_pred), RMSE(y,ridge_pred), Pearson(y,ridge_pred)

## Evaluation of each model for the 3 datasets

In [9]:
def eval_results(dataset):
    df = pd.read_csv(dataset)
    print("Evaluation Results:")
    models=["bert","elmo","gpt","gpt2","universal", "roberta","xlnet"]

    for m in models:
        print(m.upper())
        rmse_iso,rmse_lin,rmse_rid = [],[],[]
        pc_iso, pc_lin, pc_rid = [], [],[]

        for i in range(0, 1000):
            iso_rmse_score, iso_pc_score, lin_rmse_score, lin_pc_score, rid_rmse_score, rid_pc_score = evaluate_models(m,df)
            rmse_iso.append(iso_rmse_score)
            pc_iso.append(iso_pc_score)

            rmse_lin.append(lin_rmse_score)
            pc_lin.append(lin_pc_score)

            rmse_rid.append(rid_rmse_score)
            pc_rid.append(rid_pc_score)


        print("Isotonic Regression \t ==> \t RMSE :",round(avg(rmse_iso), 3)," \t Pearson Correlation :", round(avg(pc_iso), 3))
        print("Linear Regression \t ==> \t RMSE :",round(avg(rmse_lin), 3)," \t Pearson Correlation :", round(avg(pc_lin), 3))
        print("Ridge Regression \t ==> \t RMSE :",round(avg(rmse_rid), 3)," \t Pearson Correlation :", round(avg(pc_rid), 3))

In [12]:
print('Results for Dataset 1')    
eval_results('Dataset 1.csv')

Results for Dataset 1
Evaluation Results:
BERT
Isotonic Regression 	 ==> 	 RMSE : 1.072  	 Pearson Correlation : 0.314
Linear Regression 	 ==> 	 RMSE : 1.013  	 Pearson Correlation : 0.428
Ridge Regression 	 ==> 	 RMSE : 1.015  	 Pearson Correlation : 0.425
ELMO
Isotonic Regression 	 ==> 	 RMSE : 1.001  	 Pearson Correlation : 0.452
Linear Regression 	 ==> 	 RMSE : 0.961  	 Pearson Correlation : 0.525
Ridge Regression 	 ==> 	 RMSE : 0.968  	 Pearson Correlation : 0.513
GPT
Isotonic Regression 	 ==> 	 RMSE : 1.079  	 Pearson Correlation : 0.279
Linear Regression 	 ==> 	 RMSE : 1.024  	 Pearson Correlation : 0.412
Ridge Regression 	 ==> 	 RMSE : 1.021  	 Pearson Correlation : 0.418
GPT2
Isotonic Regression 	 ==> 	 RMSE : 1.093  	 Pearson Correlation : 0.226
Linear Regression 	 ==> 	 RMSE : 1.041  	 Pearson Correlation : 0.379
Ridge Regression 	 ==> 	 RMSE : 1.043  	 Pearson Correlation : 0.375
UNIVERSAL
Isotonic Regression 	 ==> 	 RMSE : 0.972  	 Pearson Correlation : 0.508
Linear Regres

In [13]:
print('Results for Dataset 2')    
eval_results('Dataset 2.csv')

Results for Dataset 2
Evaluation Results:
BERT
Isotonic Regression 	 ==> 	 RMSE : 1.478  	 Pearson Correlation : 0.563
Linear Regression 	 ==> 	 RMSE : 1.381  	 Pearson Correlation : 0.645
Ridge Regression 	 ==> 	 RMSE : 1.387  	 Pearson Correlation : 0.642
ELMO
Isotonic Regression 	 ==> 	 RMSE : 1.539  	 Pearson Correlation : 0.513
Linear Regression 	 ==> 	 RMSE : 1.421  	 Pearson Correlation : 0.615
Ridge Regression 	 ==> 	 RMSE : 1.44  	 Pearson Correlation : 0.601
GPT
Isotonic Regression 	 ==> 	 RMSE : 1.608  	 Pearson Correlation : 0.447
Linear Regression 	 ==> 	 RMSE : 1.427  	 Pearson Correlation : 0.611
Ridge Regression 	 ==> 	 RMSE : 1.452  	 Pearson Correlation : 0.593
GPT2
Isotonic Regression 	 ==> 	 RMSE : 1.617  	 Pearson Correlation : 0.432
Linear Regression 	 ==> 	 RMSE : 1.441  	 Pearson Correlation : 0.6
Ridge Regression 	 ==> 	 RMSE : 1.423  	 Pearson Correlation : 0.622
UNIVERSAL
Isotonic Regression 	 ==> 	 RMSE : 1.491  	 Pearson Correlation : 0.558
Linear Regressio

In [14]:
print('Results for Dataset 3')    
eval_results('Dataset 3.csv')

Results for Dataset 3
Evaluation Results:
BERT
Isotonic Regression 	 ==> 	 RMSE : 1.289  	 Pearson Correlation : 0.659
Linear Regression 	 ==> 	 RMSE : 1.122  	 Pearson Correlation : 0.731
Ridge Regression 	 ==> 	 RMSE : 1.074  	 Pearson Correlation : 0.769
ELMO
Isotonic Regression 	 ==> 	 RMSE : 1.067  	 Pearson Correlation : 0.771
Linear Regression 	 ==> 	 RMSE : 1.086  	 Pearson Correlation : 0.76
Ridge Regression 	 ==> 	 RMSE : 1.125  	 Pearson Correlation : 0.743
GPT
Isotonic Regression 	 ==> 	 RMSE : 1.217  	 Pearson Correlation : 0.684
Linear Regression 	 ==> 	 RMSE : 1.181  	 Pearson Correlation : 0.697
Ridge Regression 	 ==> 	 RMSE : 1.142  	 Pearson Correlation : 0.738
GPT2
Isotonic Regression 	 ==> 	 RMSE : 1.528  	 Pearson Correlation : 0.465
Linear Regression 	 ==> 	 RMSE : 1.227  	 Pearson Correlation : 0.664
Ridge Regression 	 ==> 	 RMSE : 1.195  	 Pearson Correlation : 0.7
UNIVERSAL
Isotonic Regression 	 ==> 	 RMSE : 1.126  	 Pearson Correlation : 0.738
Linear Regressio

## Final Model

In [66]:
def linear_reg_model(xTrain,yTrain):
    model = LinearRegression()
    model.fit(xTrain, yTrain)
    with open('asag_model.pkl','wb') as f:
        pickle.dump(model,f)
    with open('clipping_params.pkl', 'wb') as f:
        pickle.dump((0, 5), f)
df=pd.read_csv('Dataset 1.csv')
mdl_score = 'normalized_universal_similarity_score'
X=df[[mdl_score,'keyword_match','normalized_length_ratio']]
y=df['score_avg']
xTrain, xTest, yTrain, yTest = train_test_split(X, y, test_size=0.3, random_state=101)
linear_reg_model(xTrain,yTrain)

In [16]:
def preprocessing(q, ans):
    q = q.lower()
    ans = ans.lower()
    stop_words = set(stopwords.words('english'))
    q_res = q.translate(str.maketrans('', '', string.punctuation))
    ans_res = ans.translate(str.maketrans('', '', string.punctuation))
    q_tokens = word_tokenize(q_res)
    ans_tokens = word_tokenize(ans_res)
    demoted_tokens = [t for t in ans_tokens if t not in q_tokens]
    filtered_sent = [w for w in demoted_tokens if not w in stop_words]
    return filtered_sent

def check_tokens(sent):
    if not list:
        sent = word_tokenize(sent)
    return sent

def universal(sent):
    module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
    univ_model = hub.load(module_url)
    tokens = check_tokens(sent)
    embedding = tf.nn.l2_normalize(univ_model(tokens))
    word_arr=[]
    for i in range(len(embedding)):
        word_arr.append(embedding[i].numpy())
    return word_arr
def preprocess_lr(ans):
    stop_words = set(stopwords.words('english'))
    ans_res = ans.translate(str.maketrans('','',string.punctuation))
    ans_tokens = word_tokenize(ans_res)
    filtered_sent = [w for w in ans_tokens if not w in stop_words]
    return filtered_sent

def keyword_match(s_ans, d_ans):
    d = Rake()
    d.extract_keywords_from_text(d_ans)
    da = d.get_ranked_phrases()
    s = Rake()
    s.extract_keywords_from_text(s_ans)
    sa = s.get_ranked_phrases()
    cnt = 0
    for i in da:
        if i in sa:
            cnt += 1
    frac = cnt/len(da)
    return frac

In [133]:
with open('asag_model.pkl', 'rb') as f:
    model = pickle.load(f)
with open('clipping_params.pkl', 'rb') as f:
    clip_params = pickle.load(f)
q = input('Question: ')
ans = input('Answer: ')
model_ans = input('Desired Answer: ')
no_ans=['', ' ', 'not answered']
if ans.lower() in no_ans:
    print('Score:',0)
else:
    #Similarity score
    model_preproc = preprocessing(q, model_ans)
    stu_preproc = preprocessing(q, ans)
    if len(model_preproc) == 0:
        model_preproc = model_ans.split()
    if len(stu_preproc) == 0:
        stu_preproc = ans.split()

    model_arr=universal(model_preproc)
    stu_arr=universal(stu_preproc)
    sim_score=1-cosine(sum(model_arr),sum(stu_arr))
    sim_scores=np.concatenate((np.array([sim_score]),np.array(df['universal_similarity_score'])),axis=None)
    score_n = MinMaxScaler().fit_transform(sim_scores.reshape(-1,1))
    
    #Length Ratio
    stu_preproc=preprocess_lr(ans)
    model_preproc=preprocess_lr(model_ans)
    if len(model_preproc) == 0:
        model_preproc = model_ans.split()
    if len(stu_preproc) == 0:
        stu_preproc = ans.split()
    lr=len(stu_preproc)/len(model_preproc)
    lrs=np.concatenate((np.array([lr]),np.array(df['length_ratio'])),axis=None)
    lrs = MinMaxScaler().fit_transform(lrs.reshape(-1,1))
    
    match_val=keyword_match(ans,model_ans)
    
    y_pred = model.predict([[score_n[0],match_val, lrs[0]]])
    y_pred = np.clip(y_pred, clip_params[0], clip_params[1])

    print('Score:',y_pred[0])

Question: How does the compiler handle inline functions?
Answer: compiler ignores inline qualifier
Desired Answer: It makes a copy of the function code in every place where a function call is made.
Score: 2.564852370002992
